In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import pandas as pd
import git
import mlflow.keras
import os, sys
import numpy as np
from gensim.models.wrappers import FastText

np.random.seed(42)
import tensorflow_hub as hub
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.preprocessing import *
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
%matplotlib inline
pd.set_option('display.max_rows', None)

sys.path.append('../citation_bio_trainer')
#from feature.SpacyFeaturizer import get_spacy_feats_from_text
from feature.Featurizer import Featurizer
from util.Utils import calulate_ser_jer, load_from_folder, pad_sequences, load_embedding_matrix, evaluate, log_mlflow_results

In [3]:
path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-08~15:13:57.766608/nlp/exps/output/2020-06-08~15:13:57.766608'
#data_df = pd.read_csv(path+'/data-2020-06-08~15:14:01.185827.csv')

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## LSTM with random embedding model

In [4]:
%%time
train_data_path = '/nlp/cs_mixed_20k/cs_mixed_20k_train/'
test_data_path  = '/nlp/cs_mixed_20k/cs_mixed_20k_test/'
eval_without_intra_newline_path  = '/nlp/eval_without_intra_newline/'
eval_with_intra_newline_path     = '/nlp/eval_with_intra_newline/'

sentences_train, sent_tags_train = load_from_folder(train_data_path)
sentences_test, sent_tags_test = load_from_folder(test_data_path)
sentences_eval1, sent_tags_eval1 = load_from_folder(eval_without_intra_newline_path)
sentences_eval2, sent_tags_eval2 = load_from_folder(eval_with_intra_newline_path)

CPU times: user 1min 35s, sys: 925 ms, total: 1min 36s
Wall time: 1min 36s


In [75]:
len(sentences_eval2)

1328

In [5]:
xtrain, xvalid, ytrain, yvalid = train_test_split(sentences_train, sent_tags_train, test_size=0.1, random_state=42)

In [6]:
feat_config = {'max_vocab':100000,
               'lstm_feats':True, 
               'spacy_feats':True, 
               'google_feats': False, 
               'neural_parscit_feats': False}

In [13]:
%%time
featurizer = Featurizer(feat_config)
print("loading train ...")
train_dict, tokenizer, maxlen = featurizer.fit_transform(xtrain, ytrain)
print("loading valid ...")
valid_dict = featurizer.transform(xvalid, yvalid)
print("loading test ...")
test_dict  = featurizer.transform(sentences_test, sent_tags_test)
print("loading evals ...")
eval_dict1  = featurizer.transform(sentences_eval1, sent_tags_eval1)
eval_dict2  = featurizer.transform(sentences_eval2, sent_tags_eval2)

loading train ...
total partitions = 15
loading valid ...
total partitions = 1
loading test ...
total partitions = 1
loading evals ...
total partitions = 1
total partitions = 1
CPU times: user 9min 3s, sys: 28.5 s, total: 9min 31s
Wall time: 18min 32s


## Keras with pre-trained fast text embedding

In [76]:
model_config = {'bi-lstm':True,
                'lstm_units': 50,
                'trainable': True,
                'optimizer': Adam(
        learning_rate=0.001,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
                'output_activation' : 'sigmoid', 
                'batch_size': 256
                
                
               }

In [15]:
%%time
wiki_model = FastText.load_fasttext_format('/nlp/cc.en.300.bin')

CPU times: user 2min 18s, sys: 3.02 s, total: 2min 21s
Wall time: 2min 39s


In [22]:
%%time
embedding_matrix = load_embedding_matrix(wiki_model, feat_config['max_vocab'], tokenizer.word_index, 300)

Found 2000000 word vectors.
number of null word embeddings: 9769
CPU times: user 7.66 s, sys: 26.9 ms, total: 7.68 s
Wall time: 7.68 s


In [10]:
# %%time
# model = Sequential()
# model.add(Embedding(max_vocab, 300, weights=[embedding_matrix], trainable=True, input_length= maxlen))
# model.add(LSTM(50, return_sequences=True))
# model.add(TimeDistributed(Dense(1, activation="sigmoid")))
# # model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# model.compile(
#     optimizer=Adam(
#         learning_rate=0.001,
#         beta_1=0.0,
#         beta_2=0.0,
#         epsilon=1e-05,
#         amsgrad=False,
#     ),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )
# model.summary()

In [77]:
%%time
input = Input(shape=(maxlen,))
embed = Embedding(input_dim=feat_config['max_vocab'], input_length=maxlen, output_dim=300, weights=[embedding_matrix], trainable=model_config['trainable'])(input)
lstm = LSTM(model_config['lstm_units'], return_sequences=True)(embed)
spacy_feats = Input(shape=(maxlen,12))
conc = Concatenate([lstm, spacy_feats])
td = TimeDistributed(Dense(1, activation=model_config['output_activation']))(lstm)
model = Model(inputs=[input, spacy_feats], outputs=td)
model.compile(optimizer=model_config['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 3861)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 3861, 300)    30000000    input_8[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 3861, 50)     70200       embedding_3[0][0]                
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 3861, 12)]   0                                            
____________________________________________________________________________________________

In [78]:
class Metrics(callbacks.Callback):
    def __init__(self, train_data, validation_data):
        super(Metrics, self).__init__()
        self.validation_data = validation_data
        self.train_data = train_data
        
    def on_train_begin(self, logs={}):
        self.val_ser = []
        self.val_jer = []
        
    def on_epoch_end(self, epoch, logs):
        vx = self.validation_data[0]
        vy = self.validation_data[1]
        vx = np.array(vx)
        pred_y = None
        pred_y = self.model.predict(vx)
        py = np.argmax(pred_y, axis=-1)
        #vy = np.argmax(vy, axis=-1)
        vy = np.array(vy).flatten()
        py = np.array(py).flatten()
        ser, jer = calulate_ser_jer(vy, py)
        print(ser, jer)
        self.val_ser.append(ser)
        self.val_jer.append(jer)
        logs["val_ser"] = ser
        logs["val_jer"] = jer
        print(f"— val_ser: {ser} — val_jer: {jer}")
        return 

In [79]:
metrics = Metrics([train_dict['lstm_feats'], train_dict['labels']], [valid_dict['lstm_feats'], valid_dict['labels']])

In [ ]:
%%time
#spacy_train_arr = np.array([i.tolist() for i in train_dict['spacy_num_feats']])
#spacy_valid_arr = np.array([i.tolist() for i in valid_dict['spacy_num_feats']])
spacy_train_arr = train_dict['spacy_num_feats']
spacy_valid_arr = valid_dict['spacy_num_feats']
early_stop= EarlyStopping(monitor='val_loss',patience=3,verbose=0,mode='min',restore_best_weights=False, min_delta=0.0001)
history = model.fit([train_dict['lstm_feats'], spacy_train_arr], train_dict['labels'], verbose=1, epochs=5, batch_size= model_config['batch_size'], \
                    validation_data=([valid_dict['lstm_feats'], spacy_valid_arr], valid_dict['labels']), callbacks=[early_stop,metrics])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].title.set_text('model accuracy')
axes[0].set_ylabel('accuracy')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].title.set_text('model loss')
axes[1].set_ylabel('loss')
axes[1].set_xlabel('epoch')
axes[1].legend(['train', 'val'], loc='upper left')
fig.tight_layout()

Epoch 1/5
62/62 [==============================] - ETA: 0s - loss: 0.1038 - accuracy: 0.98021.0 1.0
— val_ser: 1.0 — val_jer: 1.0
62/62 [==============================] - 75s 1s/step - loss: 0.1038 - accuracy: 0.9802 - val_loss: 0.0193 - val_accuracy: 0.9939 - val_ser: 1.0000 - val_jer: 1.0000
Epoch 2/5
62/62 [==============================] - ETA: 0s - loss: 0.0151 - accuracy: 0.9941

In [ ]:
print('done')

### Analysis on validation data

In [29]:
%%time
valid_probs = model.predict([valid_dict['lstm_feats'], valid_dict['spacy_num_feats']])
valid_probs = valid_probs.reshape(valid_probs.shape[0], valid_probs.shape[1])
valid_preds = np.where(valid_probs > 0.5, 1, 0)

CPU times: user 9.69 s, sys: 953 ms, total: 10.6 s
Wall time: 6.5 s


In [30]:
%%time
result_valid = evaluate(valid_dict['labels'].tolist(), valid_preds.tolist())

CPU times: user 12.5 s, sys: 0 ns, total: 12.5 s
Wall time: 12.5 s


In [31]:
result_valid

{'count': 1758,
 'mean_ser': 0.004855106529219414,
 'mean_jer': 0.003997316456430747,
 'mean_acc': 0.9999484356708476,
 'num_mistakes': 266}

In [91]:
result_valid

{'count': 1758,
 'mean_ser': 0.002152679994729358,
 'mean_jer': 0.006482458698980625,
 'mean_acc': 0.9999549180436552,
 'num_mistakes': 228}

In [42]:
count_valid = 0
for ind in range(len(valid_preds)):
    pred = valid_preds[ind]
    true = valid_dict['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])

8 False positives: [45]
['Com' '.' '\n' '(' '2009' ')' '.']
10 False negatives: [1701]
['2016' '.' '\n' 'ILD' 'Concept' 'Group' ',']
13 False negatives: [449]
['Pilote' 'L.' '\n' 'Premature' 'cardiovascular' 'disease' 'following']
15 False negatives: [196]
['\n' '.' '\n' '.' '\n' '\n' 'Pluhacek']
32 False positives: [201 281 728]
['Youtube' '.' '\n' 'Available' 'at' ':'
 'https://www.youtube.com/user/CDCStreamingHealth']
['3.1' '.' '\n' '<' 'https://eu.idtdna.com/calc/analyzer' '>' '\n']
['stabilized' 'magnetic' '\n' 'skyrmions' ',' 'Nat' 'Commun']
34 False negatives: [228]
['p' '168' '\n' '(' '8)' 'Scheinfein' 'M']
40 False negatives: [357]
['of' 'human' '\n' '10' ')' 'Parkhurst' ',']
41 False negatives: [1575]
['1995;44:219–27' '.' '\n' 'Kasimanickam' 'VR' ',' 'Abdel']
44 False negatives: [1976]
['8' 'May.' '\n' 'VeriFinger' '9.0' '(' '2017']
46 False negatives: [422]
['http://www.nuctech.com' '.' '\n' 'm' ')' 'http://www' '.']
49 False negatives: [168]
['http://www.dreamweaverintl.c

### Analysis of test result

In [32]:
%%time
test_probs = model.predict([test_dict['lstm_feats'], test_dict['spacy_num_feats']])
test_probs = test_probs.reshape(test_probs.shape[0], test_probs.shape[1])
test_preds = np.where(test_probs > 0.5, 1, 0)

CPU times: user 10.3 s, sys: 1.15 s, total: 11.5 s
Wall time: 6.86 s


In [33]:
%%time
result_test = evaluate(test_dict['labels'].tolist(), test_preds.tolist())

CPU times: user 14.1 s, sys: 0 ns, total: 14.1 s
Wall time: 14.1 s


In [34]:
result_test

{'count': 1953,
 'mean_ser': 0.00448628139945021,
 'mean_jer': 0.0031924261926201407,
 'mean_acc': 0.9999553081990359,
 'num_mistakes': 276}

In [97]:
result_test

{'count': 1953,
 'mean_ser': 0.0022342707021072985,
 'mean_jer': 0.005215914626321135,
 'mean_acc': 0.9999604802472186,
 'num_mistakes': 246}

In [115]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'test'}
opt = model_config['optimizer']
model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_test, feat_config, model_config, tags)

2020/08/10 13:33:31 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/4e7aaeb1eafd4a9e8a902cd34bd54749/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


In [75]:
count = 0
for ind in range(len(test_preds)):
    pred = test_preds[ind]
    true = new_y_test_enc[ind]
    if (true == pred).all():
        pass
    else:
        count += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(new_X_test[ind])[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(new_X_test[ind])[max(0, x-3):x+4])

2 False positives: [368]
['\n' 'Moroi' '\n' 'Y.' 'Micelles' ':' '\n']
3 False negatives: [1505]
['York' '.' '\n' '.' 'http://www.weizmann.ac.il' '/' 'mcb']
4 False positives: [620]
['065105' '-' '\n' '10' '.' '\n' 'o']
6 False positives: [214]
['CO' '.' '\n' '(' '2009' ')' '.']
16 False positives: [90]
['4' '.' '\n' 'J.' 'Utt' ',' 'R.G.']
16 False negatives: [664]
['Energy' 'Council' '\n' '18' '.' 'Vassilis' 'Daioglou']
21 False positives: [74]
['13' '.' '\n' 'Stanford' ':' 'CSLI' 'pp.29']
26 False positives: [127 168]
['in' 'recent' '\n' 'years' ',' 'see' ':']
['\n' '.' '\n' 'http://www.select.cs.cmu.edu' '/' 'code' '/']
31 False negatives: [254]
[',' 'Tex' '\n' 'Huang' ',' 'A.-B.' '&']
32 False positives: [84]
[';' '2016' '\n' '[' 'accessed' '17.02.02' ']']
33 False positives: [39]
['this' 'does' '\n' 'not' 'provide' 'further' 'clarity']
34 False positives: [351]
['/' 'E8' '\n' 'M' 'Standard' 'Test' 'Methods']
37 False positives: [606]
['Inf' '.' '\n' 'Model' '.' '2016;56:1609–14' '.

### Evaluation of eval dataset

In [35]:
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/e42c50d933ff49fb826dfe5874fa6d51/artifacts/models")

In [36]:
eval_probs1 = model.predict([eval_dict1['lstm_feats'], eval_dict1['spacy_num_feats']])
eval_probs1 = eval_probs1.reshape(eval_probs1.shape[0], eval_probs1.shape[1])
eval_preds1 = np.where(eval_probs1 > 0.5, 1, 0)
result_eval1 = evaluate(eval_dict1['labels'].tolist(), eval_preds1.tolist())

In [42]:
eval_probs2 = model.predict([eval_dict2['lstm_feats'], eval_dict2['spacy_num_feats']])
eval_probs2 = eval_probs2.reshape(eval_probs2.shape[0], eval_probs2.shape[1])
eval_preds2 = np.where(eval_probs2 > 0.5, 1, 0)
result_eval2 = evaluate(eval_dict2['labels'].tolist(), eval_preds2.tolist())

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


In [43]:
result_eval2

{'count': 1328,
 'mean_ser': 0.38666973209478767,
 'mean_jer': 0.042736742169992036,
 'mean_acc': 0.9913145199289778,
 'num_mistakes': 1313}

In [54]:
count_valid = 0
for ind in range(20):
    pred = eval_preds1[ind]
    true = eval_dict1['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 False positives: [   1    2    3 ... 3858 3859 3860]
['Anderson' ',' 'J.L.' '(' '1997']
['Anderson' ',' 'J.L.' '(' '1997' ')']
['Anderson' ',' 'J.L.' '(' '1997' ')' '.']
[',' 'J.L.' '(' '1997' ')' '.' 'The']
['J.L.' '(' '1997' ')' '.' 'The' 'impact']
['(' '1997' ')' '.' 'The' 'impact' 'of']
['1997' ')' '.' 'The' 'impact' 'of' 'dynamical']
[')' '.' 'The' 'impact' 'of' 'dynamical' 'constraints']
['.' 'The' 'impact' 'of' 'dynamical' 'constraints' 'on']
['The' 'impact' 'of' 'dynamical' 'constraints' 'on' 'the']
['impact' 'of' 'dynamical' 'constraints' 'on' 'the' 'selection']
['of' 'dynamical' 'constraints' 'on' 'the' 'selection' 'of']
['dynamical' 'constraints' 'on' 'the' 'selection' 'of' 'initial']
['constraints' 'on' 'the' 'selection' 'of' 'initial' 'conditions']
['on' 'the' 'selection' 'of' 'initial' 'conditions' 'on']
['the' 'selection' 'of' 'initial' 'conditions' 'on' 'ensemble']
['selection' 'of' 'initial' 'conditions' 'on' 'ensemble' 'predictions']
['of' 'initial' 'conditions' 'on

KeyboardInterrupt: 

In [48]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'eval_with_intra_newline_path'}
#opt = model_config['optimizer']
#model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_eval2, feat_config, model_config, tags)

2020/08/11 11:11:59 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/0ff4590506a84551a553c1a52221802c/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


In [73]:
sentences_train[0][0:500]

'1 ) J. Chang et al . , “ 12.1 A 7 nm 256Mb SRAM in high - k metal - gate FinFET technology with write - assist circuitry for low - VMIN applications , ” 2017 IEEE International Solid - State Circuits Conference ( ISSCC ) , San Francisco , CA , 2017 , pp . 206 - 207 . \n 2 ) T. Standaert et al . , “ BEOL process integration for the 7 nm technology node , ” 2016 IEEE International Interconnect Technology Conference / Advanced Metallization Conference ( IITC / AMC ) , San Jose , CA , 2016 , pp . 2 -'

In [53]:
sentences_eval2[0:5]

["Aguado , \n T. \n , Romero \n , \n E. , Monory , K. , Palazuelos , J. , Sendtner , M. , Marsicano , G. , Lutz , B. , Guzmán , M. , Galve - Roperh , I. ( 2007 ) The CB1 cannabinoid receptor mediates excitotoxicity - induced neural progenitor proliferation and neurogenesis . J. Biol . Chem . 282 , 23892‑23898 . \n Alonso , A. \n & Kohler , \n C. ( 1984 ) A study \n of \n the reciprocal connections between the septum and entorhinal area using anterograde and retrograde axonal transport methods in the rat brain . J. Comp . Neurol . 225 , 327 - 343 . \n Arabadzisz , \n D. , Antal \n , K. , \n Parpan , F. , \n Emri , Z. , Fritschy , J.M. ( 2005 ) Epileptogenesis and chronic seizures in a mouse model of temporal lobe epilepsy are associated with distinct EEG patterns and selective neurochemical alterations in the contralateral hippocampus . Exp . Neurol . 194 , 76‑90 . \n Ashwood , \n T.J. \n & Wheal , H.V. ( \n 1986 \n ) Loss of inhibition in the CA1 region of the kainic acid lesioned hipp

In [52]:
#sentences_train[0]